## Atmospheric River Detection Testing

In [1]:
import xarray as xr
import hvplot.xarray
import hvplot.pandas

from config import ar_params, ard_fp
from ar_detection import *

In [2]:
ard_fp

PosixPath('/atlas_scratch/cparr4/atmospheric_rivers/inputs/ar_detection_inputs.nc')

Check AR detection parameters imported from ```config.py```. See the ```config.py``` for more detailed description of these parameters.

In [3]:
ar_params

{'window': 75,
 'ivt_percentile': 85,
 'ivt_floor': 100,
 'direction_deviation_threshold': 45,
 'mean_meridional': 50,
 'orientation_deviation_threshold': 45,
 'min_axis_length': 1000}

**Note:** we are using 1000km for the minimum AR length during this test, since using 2000km from Guan & Waliser would require downloading a much larger spatial domain. Also, the 2:1 minimum length to width ratio has been hashed out in the ```filter_regions_by_geometry()``` function; without the full length of the AR to compute this criteria, some AR events were being dropped erroneously. 

For this testing phase, a bounding box of [60, -150, 50, -130] was used. Once development of these functions is complete, a larger spatial domain should be downloaded and the parameters/functions mentioned above should be updated.

Open the dataset resulting from the ```compute_ivt.py``` script, using the ```ard_fp``` variable imported from ```config.py```. Alternatively, reset the ```ard_fp``` variable to an explicit test dataset.

In [16]:
# use test dataset with time=(slice('2020-01-01', '2021-06-01') instead of full dataset
# normal percentile in this case was only computed for this time slice and will be very similar to seasonal percentile
# ard_fp = '/atlas_scratch/jdpaul3/era5/inputs/ard_test.nc'

In [6]:
ds = xr.open_dataset(ard_fp)
ds.close()

Subset one month around a known atmospheric river event (Haines 2020).

In [5]:
t1 = '2020-11-15'
t2 = '2020-12-15'

ds = ds.sel(time=slice(t1, t2))

Compute intensity mask and create new variable in the dataset.

In [20]:
ds['intensity_mask'] = intensity_mask(ds['ivt_mag'], ds['ivt_pctile_n'], ar_params['ivt_floor'])

Label regions and create new variable in the dataset.

In [21]:
ds['regions'] = label_regions(ds['intensity_mask'])

Filter regions by geometric criteria. (This function alters the dataset in place; the output dictionary is just a reference used for confirming bad labels are dropped.)

In [22]:
geo_drop_dict = filter_regions_by_geometry(ds['regions'], ar_params['min_axis_length'])

### plotting

Check out IVT magnitude, IVT percentile, and labeled AR regions in two interactive viewers. The Haines AR event was on Dec 1 and 2, 2020.

In [47]:
ds['ivt_mag'].hvplot(groupby='time', x = 'longitude', y = 'latitude', width=600, widget_type='scrubber', widget_location='right', clim=(0, 800))

Row
    [0] HoloViews(DynamicMap, widget_location='right', widget_type='scrubber')
    [1] Column
        [0] VSpacer()
        [1] WidgetBox
            [0] Player(end=123, width=550)
        [2] VSpacer()

In [45]:
ds['regions'].hvplot(groupby='time', x = 'longitude', y = 'latitude', width=600, widget_type='scrubber', widget_location='right', clim=(0, 4))

Row
    [0] HoloViews(DynamicMap, widget_location='right', widget_type='scrubber')
    [1] Column
        [0] VSpacer()
        [1] WidgetBox
            [0] Player(end=123, width=550)
        [2] VSpacer()

### unfinished testing

Testing of ```filter_regions_by_direction()``` function. This is incomplete and currently just prints a list of ```ivt_dir``` values for each labeled region. See ```ar_detection.py``` for more details about what this function should eventually accomplish.

In [13]:
dir_drop_dict = filter_regions_by_direction(ds['regions'], ds['ivt_dir'])

Labeled regions at 2020-11-21T00:00:00.000000000: [0.0, 1.0]
Mean IVT directions at 2020-11-21T00:00:00.000000000: [233.89469668 282.77285714]
[267, 260, 258, 254, 252, 248, 247, 247, 248, 262, 248, 248, 247, 245, 240, 233, 225, 215, 206, 248, 197, 263, 266, 222, 227, 224, 217, 242, 247, 251, 255, 264, 258, 264, 267, 269, 274, 272, 270, 269, 267, 261, 218, 189, 180, 250, 253, 257, 260, 263, 266, 269, 270, 246, 268, 265, 263, 262, 261, 259, 257, 255, 252, 266, 184, 242, 213, 175, 170, 165, 160, 156, 154, 154, 154, 236, 154, 156, 159, 172, 187, 205, 213, 219, 215, 154, 196, 179, 160, 142, 140, 139, 137, 142, 152, 181, 214, 144, 230, 230, 224, 216, 247, 252, 256, 259, 263, 234, 265, 148, 160, 258, 259, 258, 257, 253, 249, 243, 235, 153, 226, 208, 200, 194, 190, 185, 180, 176, 168, 217, 268, 270, 274, 221, 211, 201, 194, 189, 184, 179, 175, 230, 168, 156, 152, 149, 149, 148, 146, 146, 150, 162, 238, 244, 250, 272, 271, 269, 268, 267, 266, 264, 263, 260, 257, 254, 252, 252, 253, 253, 253, 2

NameError: name 'dirs_values_list' is not defined